In [2]:
import sys
sys.path.append('../python_packages_static')
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import fiona
from shapely.geometry import shape
from gisutils import project
from pydrograph import Nwis
import geopandas as gp
import requests

ModuleNotFoundError: No module named 'requests'

# Notebook to extract NWIS data from model area and make obs tables

## 1. Get DV surface water obs from USGS StreamStats: https://streamstats.usgs.gov/ss/
### 1.1 Get streamflow daily values sites
**Create model bounding box and retrieve NWIS streamflow DV sites** 

uses the excellent `pydrogrpah`: https://github.com/aleaf/pydrograph

In [ ]:
extent_shp = '../source_data/Shapefiles/Extents/Model_Extent_HUC12.shp'
epsg = 5070

In [ ]:
extent_poly = shape(fiona.open(extent_shp).next()['geometry'])
extent_poly_ll = project(extent_poly, "+init=epsg:{}".format(epsg), "+init=epsg:4269")
extent_poly_ll.bounds

In [ ]:
bound = gp.read_file(extent_shp)

In [ ]:
nwis = Nwis(extent=extent_poly_ll)

**Get streamflow daily values sites using `pydrograph`**

In [ ]:
all_dvs = nwis.get_siteinfo('daily_values')

In [ ]:
all_dvs

In [ ]:
bound = bound.to_crs(epsg=4269)

fig, ax = plt.subplots()
bound.plot(ax=ax, facecolor='None', edgecolor='black')
ax.scatter(all_dvs.dec_long_va, all_dvs.dec_lat_va)
plt.show()

### Only SANDBURG CREEK AT ELLENVILLE NY (01366650) and NEVERSINK RIVER AT WOODBOURNE NY (01436500) can be used for flux targets
* NEVERSINK RIVER AT NEVERSINK NY at edge of model, used for SFR inflow (see 0.7_make_SFR_inflow.ipynb)
* Data collection at GUMAER BROOK NEAR WURTSBORO NY only started on 2019-12-11, too little data avialable for annual average flow/BFI

### 1.2 Extract Mean annual flow and BFI data from Streamstats: https://streamstats.usgs.gov/ss/
**gage pages:** 
* SANDBURG CREEK AT ELLENVILLE NY (01366650): https://streamstatsags.cr.usgs.gov/gagepages/html/01366650.htm
* NEVERSINK RIVER AT WOODBOURNE NY (01436500): https://streamstatsags.cr.usgs.gov/gagepages/html/01436500.htm

In [ ]:
gages = ['01366650', '01436500']

sites_dict = {}

In [ ]:
# read from gage streamstats pages

for gage in gages: 
    site_dict = {}
    url = f'https://streamstatsags.cr.usgs.gov/gagepages/html/{gage}.htm'
    info = ['Mean_Annual_Flow', 'Average_BFI_value', 'Latitude (degrees NAD83)', 'Longitude (degrees NAD83)']

    r = requests.get(url, stream=True)
    lines = [l.decode('cp1252') for l in r.iter_lines()]

    line_nums = [] 

    for i, line in enumerate(lines):
        for t in info:
            if t in line: 
                line_nums.append(i)
                line_nums.append(i+1)
    for prop, val in zip(line_nums[::2], line_nums[1::2]):
        p = lines[prop].replace('<td>','').replace('</td>','').replace("<tr class='even'><td width='197'>",'').replace("<tr><td width='200'>",'')
        v = float(lines[val].replace('<td>','').replace('</td>','').replace("<td width='590'>",'').replace('</tr>',''))
        print(f'  prop: {p}')
        print(f'  val: {v}')
        site_dict[p] = v
    print('\n')
    sites_dict[gage] = site_dict

In [ ]:
df = pd.DataFrame.from_dict(sites_dict).T.reset_index(drop=False).rename(columns={'index':'site_id', 'Mean_Annual_Flow':'Mean_Annual_Flow_cfs'})
df

In [ ]:
# write out to processed data
df.to_csv('../processed_data/NWIS_DV_STREAMSTATS_INFO.csv', index=False)

## 2. Get groundwater daily values

In [ ]:
gw_dv = nwis.get_siteinfo('gwdv')
gw_dv

### Pull groundwater data for the lone GW DVs site in the model domain 
uses `pydrograph`: https://github.com/aleaf/pydrograph

In [ ]:
gw_site = gw_dv.iloc[0]['site_no']

# pydrograh gw dv data retrieval not working at the momnet. do it manually for now -- see below:

gw_data = nwis.get_all_dvs([gw_site], 
                           parameter_code='72019',
                           start_date='2009-01-01',
                           end_date='2016-01-01'
                          )

In [ ]:
gw_df = gw_data[gw_site]
gw_df = gw_df.groupby('site_no').mean()
gw_df = gw_df.rename(columns={'106190_72019_00003':'ave_dtw_ft'})
gw_df = gw_df.join(gw_dv[['alt_va']])
gw_df['gw_elev_ft'] = gw_df['alt_va'] - gw_df['ave_dtw_ft']
gw_df['gw_elev_m'] = gw_df['gw_elev_ft'] * 0.3048
gw_df.to_csv('../processed_data/NWIS_GW_DV_data.csv')
gw_df

### export NWIS gw sites table for obs section of `neversink_full.yml`

In [ ]:
gw_dv_gdf = gp.GeoDataFrame(gw_dv, crs="EPSG:4269", geometry=gw_dv.geometry)
gw_dv_gdf_reproj = gw_dv_gdf.to_crs(epsg=epsg)
gw_dv_gdf_reproj['x'] = gw_dv_gdf_reproj.geometry.x
gw_dv_gdf_reproj['y'] = gw_dv_gdf_reproj.geometry.y
gw_dv_gdf_reproj['obsprefix'] = gw_dv_gdf_reproj.index
gw_dv_gdf_reproj

In [ ]:
gw_dv_gdf_reproj.to_csv('../processed_data/NWIS_GW_DV_sites.csv', index=False)